Importing pandas ; Importing numpy ; Importing seaborn ; Importing matplotlib ; Importing monthly progress prediction by book value

In [91]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
df_progress=pd.read_csv('../data/output/monthly-progress-by-book-value.csv',sep=';')

Creating the mean columns of progress dataframe

In [92]:
years=df_progress['Year'].unique()
prediction_interval=range(2,6)

df_progress = df_progress.set_index(['Year', 'Month'])

for year in years:
    index_year,index_month=df_progress.index.get_level_values('Year'),df_progress.loc[year].index.get_level_values('Month')
    for interval in prediction_interval:
        if year-interval<min(years):
            series_mean=np.nan
        else :
            series_mean=df_progress[(index_year<year)&(index_year>=year-interval)].groupby('Month')['Progress'].mean()
            for month in index_month:
                df_progress.loc[(year,month),str(interval)+' Mean']=series_mean[month]

       Progress    2 Mean    3 Mean    4 Mean    5 Mean
Month                                                  
1      0.863482  0.839685  0.832720  0.827769  0.827241
2      0.883936  0.867465  0.854999  0.856713  0.858184
3      0.884467  0.872095  0.862514  0.869888  0.873431
4      0.895450  0.901418  0.874504  0.884056  0.892915
5      0.900054  0.901556  0.873967  0.871020  0.885478
6      0.944548  0.901766  0.872579  0.864472  0.878072
7      0.982507  0.925514  0.912255  0.891598  0.905807
8      0.969029  0.964249  0.958048  0.918304  0.932573
9      0.970365  0.976999  0.969797  0.914060  0.939638
10     0.992446  0.997143  0.993421  0.947482  0.974090
11     1.013026  1.033052  0.999383  0.953190  0.982727
12     1.045014  1.062125  1.016284  0.967582  0.996161


Visualizing the actual progress and the mean progress